In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt

*Setup logging*

In [2]:
import logging
from logging import StreamHandler
import sys

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.handlers = []
logger.addHandler(StreamHandler(sys.stdout))

*Setup open and close functions*

In [3]:
from selenium.webdriver.chrome.options import Options as DriveOptions
from selenium.webdriver.chrome.service import Service as DriveService
from selenium.webdriver import Chrome as ChromeDriver


def is_driver_created() -> bool:
    if "driver" in globals():
        driver = globals()["driver"]
        if driver is not None and isinstance(driver, ChromeDriver):
            return True
    return False


def setup_connection() -> None:
    global driver
    if is_driver_created():
        return
    chrome_driver_path = "D:\\Program Files\\ChromeDriver\\chromedriver.exe"
    chrome_options = DriveOptions()
    #chrome_options.add_argument("--headless")
    service = DriveService(executable_path=chrome_driver_path)
    driver = ChromeDriver(service=service, options=chrome_options)


def close_connection() -> None:
    global driver
    if not is_driver_created():
        return
    driver.quit()
    del driver
    logger.info("connection closed")

close_connection()


In [4]:
from time import sleep
from typing import List
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from decimal import Decimal


def convert_money_to_decimal(text: str) -> Decimal:
    text = text.replace("R$ ", "").replace(",", ".")
    return Decimal(text if text else 0)


class RelvaVerde:
    _url = "https://www.lojarelvaverde.com.br/bebida-de-castanha-de-caju-sabor-leite-sem-leite-possible-1l-p3123"

    _selectors = {
        "price_label": "#product-details > b > div > div.procuct-detail > div.product-values > div > div.price > b",
        "buy_button": "#product-details > b > div > div.procuct-detail > b > div.botao_comprar > div > a > button",
        "front_qnt_input": "#quantity",
        "cart_qnt_input": "#cart-items > #item-3123 * input",
        "check_cart_button": ".fancybox-content > .popup-product-added * div.actions.flex > button.w-100.btn-primary.btn.cart",
        "open_cart_button": ".sidecart-button",
        "postcode_input": "#cep",
        "postcode_search_button": "#btn-calc-frete > i > svg > path",
        "shipping_price_labels": "#side-cart > div.cart-total.flex > div.shopping-cart-shipping.flex.w-100 > dl > dl.shipping-options > .option > * > .price",
        "reject_cookie_button": "#adopt-reject-all-button",
    }

    def __init__(self, driver: ChromeDriver) -> None:
        self._driver = driver
        self._elements = dict.fromkeys(self._selectors.keys())
        self._data = dict()

    @staticmethod
    def _get_domain_from_url(url: str) -> str:
        parts = url.split("/")
        return "/".join(parts[:3])

    def _load_url(self) -> None:
        self._driver.get(self._url)
        site = self._driver.current_url.split("/")
        site = "/".join(site[:3])
        logger.debug(f"site loaded {self._get_domain_from_url(site)}")

    def _get_element(self, key: str) -> WebElement:
        if key in self._elements:
            element = self._elements[key]
        if not isinstance(element, WebElement):
            element = self._elements[key] = self._wait_element(key)
        return element

    def _wait_element(
        self, key: str, max_tries: int = 10, max_seconds: float = 10
    ) -> WebElement:
        for _ in range(max_tries):
            sleep(max_seconds / max_tries)
            try:
                element = self._driver.find_element(
                    By.CSS_SELECTOR, self._selectors[key]
                )
                return element
            except NoSuchElementException:
                continue
        raise NoSuchElementException

    def _get_elements(self, key: str) -> List[WebElement]:
        if key in self._elements:
            element = self._elements[key]
        if not isinstance(element, WebElement):
            element = self._elements[key] = self._driver.find_elements(
                By.CSS_SELECTOR, self._selectors[key]
            )
        return element

    def _click_scrolled_element(self, element: WebElement, max_tries: int = 10):
        actions = ActionChains(self._driver)
        actions.scroll_from_origin(ScrollOrigin.from_viewport(), 0, 0).perform()
        for _ in range(max_tries):
            if element.is_displayed():
                break
            actions.send_keys(Keys.PAGE_DOWN).perform()
        actions.click(element).perform()

    def _click_button(self, key: str) -> None:
        button = self._get_element(key)
        self._click_scrolled_element(button)

    def _set_input(self, key: str, value: str) -> None:
        input = self._get_element(key)
        self._click_scrolled_element(input)
        sleep(2)    #@TODO bug, when setting the qnt value, the product is being deleted
        input.clear()
        input.send_keys(value)

    def _read_element(self, key: str) -> str:
        label = self._get_element(key)
        return label.text

    def _get_price(self) -> Decimal:
        text = self._read_element("price_label")
        return convert_money_to_decimal(text)

    def _set_qnt(self, qnt: int) -> None:
        self._set_input("front_qnt_input", str(qnt))

    def _press_buy(self) -> None:
        self._click_button("buy_button")

    def _is_shipping_tab_open(self) -> bool:
        input = self._get_element("postcode_input")
        return input.is_displayed()

    def _open_shipping_tab(self, max_tries: int = 10, max_seconds: float = 10) -> None:
        actions = ActionChains(self._driver)
        for _ in range(max_tries):
            if self._is_shipping_tab_open():
                return
            self._click_button("open_cart_button")
            for _ in range(5):
                try:
                    actions.move_to_element(self._get_element("postcode_input")).perform()
                except MoveTargetOutOfBoundsException:
                    sleep(0.1)
            sleep(max_seconds / max_tries)
        raise Exception("Open shipping tab failed")

    def _press_check_cart(self) -> None:
        self._click_button("check_cart_button")

    def _collect_shipping_costs(self) -> List[Decimal]:
        labels = self._get_elements("shipping_price_labels")
        return [convert_money_to_decimal(l.text) for l in labels]

    def _calculate_shipping_first(self, postcode: str) -> List[Decimal]:
        self._open_shipping_tab()
        if self._read_element("postcode_input") != postcode:
            self._set_input("postcode_input", postcode)
        self._click_button("postcode_search_button")
        self._open_shipping_tab()
        # sleep(.5)
        return self._collect_shipping_costs()

    def _calculate_shipping(self, qnt: int) -> List[Decimal]:
        self._open_shipping_tab()
        qnt_text = self._read_element("cart_qnt_input")
        self._open_shipping_tab()
        if (int(qnt_text) if qnt_text else 0) != qnt:
            sleep(1)
            self._set_input("cart_qnt_input", str(qnt))
            sleep(5)
        self._click_button("postcode_search_button")
        self._open_shipping_tab()
        # sleep(.5)
        return self._collect_shipping_costs()

    def process(self) -> dict:
        self._driver.maximize_window()
        if self._driver.current_url != self._url:
            self._load_url()
        try:
            self._click_button("reject_cookie_button")
        except NoSuchElementException:
            pass
        self._data["unit_price"] = self._get_price()
        shipping_costs = {}
        for i in range(1, 4 + 1):
            qnt = 6 * i
            if i == 1:
                self._set_qnt(qnt)
                self._press_buy()
                self._press_check_cart()
                costs = self._calculate_shipping_first("28990-772")
            else:
                costs = self._calculate_shipping(qnt)
            shipping_costs[qnt] = costs
        self._data["shipping_costs"] = shipping_costs
        return self._data

*Excute my code*

In [5]:
setup_connection()


def execute():
    try:
        p = RelvaVerde(driver)
        data = p.process()
        print(data)
        # display(pd.DataFrame.from_dict(data, 'index', columns=['values']))
    finally:
        pass  # close_connection(driver)


execute()

site loaded https://www.lojarelvaverde.com.br


Exception: Open shipping tab failed